In [1]:
import catboost
import pandas as pd

In [2]:
# read limited nrows to do it faster
df = pd.read_csv('df_multiclass_temperature_latlons.csv', nrows=1000000)
df.head()

,fact_time,fact_temperature,fact_cwsm_class,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,cmc_0_0_0_1000,cmc_0_0_0_2_grad,cmc_0_0_0_2_interpolated,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1546235640,-1.0,0.0,233.0,-69.005015,-2.100714,744.142216,273.668750,-1.026855,270.473242,...,0.000000,0.00000,0.0,0.0,0.0,0.0000,0.600006,-3.049994,0.0,0.0
1,1546246800,-4.0,0.0,1115.0,18.247833,-5.650000,674.481747,276.918750,-6.305103,267.294434,...,0.000000,0.00000,0.0,0.0,0.0,0.0000,0.000000,-20.049994,0.0,0.0
2,1546257600,23.0,21.0,100.0,39.512251,24.720714,752.775041,297.457642,2.026587,297.439331,...,0.000000,0.00000,0.0,0.0,0.0,0.0000,-0.699982,23.650018,2.0,3.0
3,1546276500,-1.0,20.0,369.0,20.003954,-12.332143,730.694295,268.806470,1.895337,270.848641,...,0.219312,2.56235,0.0,0.0,0.0,0.0015,1.500000,-1.549994,-1.0,92.0
4,1546279200,32.0,20.0,14.0,-2.483327,28.745000,744.973269,303.427393,-2.818262,300.251953,...,0.000000,0.00000,0.0,0.0,0.0,0.0000,-0.716553,21.350000,-1.0,0.0


In [3]:
df.shape

(1000000, 126)

In [4]:
# three first columns are timestamp, fact_temperature and fact_cwsm_class
X = df.iloc[:700000, 3:-1]
y_multiclass = df['fact_cwsm_class'][:700000]
y_temperature = df['fact_temperature'][:700000]

X_test = df.iloc[700000:,3:-1]
y_multiclass_test = df['fact_cwsm_class'][700000:]
y_temperature_test = df['fact_temperature'][700000:]

In [5]:
model_multiclass = catboost.CatBoostClassifier(
    iterations=1000,
    depth=3,  
    loss_function='MultiClass',
    eval_metric='Accuracy')

In [6]:
model_multiclass.fit(X, y_multiclass, verbose=50, eval_set=(X_test, y_multiclass_test))

Learning rate set to 0.123092
0:	learn: 0.4703471	test: 0.4863333	best: 0.4863333 (0)	total: 411ms	remaining: 6m 50s
50:	learn: 0.5406629	test: 0.5407300	best: 0.5407300 (50)	total: 16.3s	remaining: 5m 2s
100:	learn: 0.5517543	test: 0.5476733	best: 0.5476733 (100)	total: 31.3s	remaining: 4m 38s
150:	learn: 0.5585486	test: 0.5521533	best: 0.5521533 (150)	total: 45.8s	remaining: 4m 17s
200:	learn: 0.5636829	test: 0.5550933	best: 0.5551733 (199)	total: 1m	remaining: 3m 59s
250:	learn: 0.5673629	test: 0.5565300	best: 0.5565400 (247)	total: 1m 14s	remaining: 3m 43s
300:	learn: 0.5697786	test: 0.5582000	best: 0.5582867 (298)	total: 1m 29s	remaining: 3m 27s
350:	learn: 0.5724029	test: 0.5589567	best: 0.5590833 (338)	total: 1m 43s	remaining: 3m 12s
400:	learn: 0.5741814	test: 0.5597600	best: 0.5597600 (400)	total: 1m 58s	remaining: 2m 56s
450:	learn: 0.5759629	test: 0.5604367	best: 0.5605100 (449)	total: 2m 13s	remaining: 2m 41s
500:	learn: 0.5775986	test: 0.5610700	best: 0.5612033 (496)	total

In [7]:
model_temperature = catboost.CatBoostRegressor(
    iterations=1000,
    depth=3,
    eval_metric='RMSE')

model_temperature.fit(X, y_temperature, verbose=50, eval_set=(X_test, y_temperature_test))

Learning rate set to 0.158932
0:	learn: 9.6369386	test: 12.3291732	best: 12.3291732 (0)	total: 59.7ms	remaining: 59.7s
50:	learn: 2.5765855	test: 2.7542641	best: 2.7542641 (50)	total: 2.66s	remaining: 49.5s
100:	learn: 2.3991266	test: 2.5919279	best: 2.5919279 (100)	total: 5.31s	remaining: 47.3s
150:	learn: 2.3227279	test: 2.5203140	best: 2.5203140 (150)	total: 7.88s	remaining: 44.3s
200:	learn: 2.2743056	test: 2.4781697	best: 2.4781697 (200)	total: 10.5s	remaining: 41.8s
250:	learn: 2.2418797	test: 2.4510107	best: 2.4510107 (250)	total: 13.1s	remaining: 39.1s
300:	learn: 2.2151919	test: 2.4282560	best: 2.4282560 (300)	total: 15.7s	remaining: 36.4s
350:	learn: 2.1954724	test: 2.4126463	best: 2.4126463 (350)	total: 18.2s	remaining: 33.7s
400:	learn: 2.1787387	test: 2.4001163	best: 2.4001163 (400)	total: 20.9s	remaining: 31.2s
450:	learn: 2.1656923	test: 2.3910880	best: 2.3910880 (450)	total: 23.3s	remaining: 28.4s
500:	learn: 2.1521877	test: 2.3804420	best: 2.3804420 (500)	total: 26s	re